# Pinterest Data Pipeline
This is for the README File in the future. May contain additional notes for myself wherever I have problems.

#### Key Pair
Creating the keypar was our first course of action using the provided Account ID, IAM user name and Password to connect to the AWS Account and navigate through the console to locate the appropriate **KeyPairId**.
</br>
See file [0e2a0bfcc015-key-pair.pem](0e2a0bfcc015-key-pair.pem)

#### Connect to EC2
ssh -i "0e2a0bfcc015-key-pair.pem" ec2-user@ec2-184-73-75-230.compute-1.amazonaws.com

In [ ]:
ssh -i "0e2a0bfcc015-key-pair.pem" ec2-user@ec2-184-73-75-230.compute-1.amazonaws.com

Then ran code 

In [ ]:
sudo yum update

You can then use the standard command line commands to navigate around the file directory. <br>
Also, you can use "nano" to create and open files.

#### Setting Up Kafka
first ran: sudo yum update
<br> then installed Java using: sudo yum install java-1.8.0-openjdk.x86_64 java-1.8.0-openjdk-devel.x86_64

<br> Kafka Target
<br> https://archive.apache.org/dist/kafka/2.8.1/kafka_2.12-2.8.1.tgz
</br>
May consider following, https://www.digitalocean.com/community/tutorials/how-to-install-apache-kafka-on-ubuntu-20-04


In [ ]:
wget https://archive.apache.org/dist/kafka/2.8.1/kafka_2.12-2.8.1.tgz

#### Created CLIENT PROPERTIES file inside of the EC2 > Kafka

In [ ]:
# Sets up TLS for encryption and SASL for authN.
security.protocol = SASL_SSL

# Identifies the SASL mechanism to use.
sasl.mechanism = AWS_MSK_IAM

# Binds SASL client implementation.
sasl.jaas.config = software.amazon.msk.auth.iam.IAMLoginModule required awsRoleArn=arn:aws:iam::584739742957:role/0e2a0bfcc015-ec2-access-role;

# Encapsulates constructing a SigV4 signature based on extracted credentials.
# The SASL client bound by "sasl.jaas.config" invokes this class.
sasl.client.callback.handler.class = software.amazon.msk.auth.iam.IAMClientCallbackHandler

#### BOOTSTRAP SERVER > Private Endpoint

In [ ]:
b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098

#### Plaintext Apache Zookeeper connection string

In [ ]:
z-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:2181,z-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:2181,z-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:2181

#### Create Kafka Topic
To do this, while located inside the Kafka_Folder/bin use the below code.

In [ ]:
./kafka-topics.sh --bootstrap-server BootstrapServerString --command-config client.properties --create --topic <topic_name>

replacing the BootstrapServerString for the previously listed BOOTSTRAP SERVER PRIVATE ENDPOINT.

PINTEREST POST DATA

In [ ]:
./kafka-topics.sh --bootstrap-server b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --command-config client.properties --create --topic 0e2a0bfcc015.pin

POST GEO LOCATION

In [ ]:
./kafka-topics.sh --bootstrap-server b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --command-config client.properties --create --topic 0e2a0bfcc015.geo

POST USER DATA

In [ ]:
./kafka-topics.sh --bootstrap-server b-1.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-3.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098,b-2.pinterestmskcluster.w8g8jt.c12.kafka.us-east-1.amazonaws.com:9098 --command-config client.properties --create --topic 0e2a0bfcc015.user